In [53]:
# imports
%matplotlib inline
# %pylab osx
import tensorflow as tf
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
from scipy.misc import imresize
from skimage.transform import resize
# Some additional libraries which we'll use just
# to produce some visualizations of our training
from libs.utils import montage
from libs import gif
import IPython.display as ipyd
plt.style.use('ggplot')
import struct
import re
from PIL import Image
import gzip
import codecs

import math
import random

# Bit of formatting because I don't like the default inline code style:
from IPython.core.display import HTML
HTML("""<style> .rendered_html code { 
    padding: 2px 4px;
    color: #c7254e;
    background-color: #f9f2f4;
    border-radius: 4px;
} </style>""")

In [7]:
def match_images(files, input_dir):
    for i in range(0, len(files)):
        print (i)
        img1 = plt.imread(os.path.join(input_dir, files[i]))
        img1 = (img1 * 255).round().astype(np.uint8)
        img1 = imresize(img1, (64,64))
        for j in range(i+1 , len(files)):
            if (files[j][:4] == files[i][:4]):
                name = "match"+str(files[i][:10]) + "_"+ str(files[j][:10]) + ".png"
                img2 = plt.imread(os.path.join(input_dir, files[j]))
                img2 = (img2 * 255).round().astype(np.uint8)
                img2 = imresize(img2, (64,64))
                #img = img1 & img2
                img = np.vstack( (img1,img2))
                plt.imsave(os.path.join('/Users/nikhilshekhar/deep_learning_working_directory/data/', name),img)

In [8]:
def non_match_images(files, input_dir):
    for i in range(1, 1569):
        if(i < 10):
            key = "000" + str(i)
        elif (i < 100):
            key = "00" + str(i)
        elif (i <1000):
            key = "0" + str(i)
        else :
            key = str(i)
        shortlisted_file_names = [ filename for filename in files if filename[:4] == key]
        print(i)
        if shortlisted_file_names:
            shortlisted = list(set(files) - set(shortlisted_file_names))
            for j in range(0,54):
                img1_key = random.choice(shortlisted_file_names)
                #print img1_key
                img1 = plt.imread(os.path.join(input_dir, img1_key))
                img1 = (img1 * 255).round().astype(np.uint8)
                img1 = imresize(img1, (64,64))
                img2_key = random.choice(shortlisted)
                shortlisted = list(set(shortlisted) - set(img2_key))
                img2 = plt.imread(os.path.join(input_dir, img2_key))
                name = "mis_match" + str(img1_key[:10]) + "_" + str(img2_key[:10]) + ".png"
                img2 = (img2 * 255).round().astype(np.uint8)
                img2 = imresize(img2, (64,64))
                img = np.vstack( (img1,img2)) 
                plt.imsave(os.path.join('/Users/nikhilshekhar/deep_learning_working_directory/data/', name),img)


In [10]:
input_dir = '/Users/nikhilshekhar/deep_learning_working_directory/images/'
files = [ input_file for input_file in os.listdir(input_dir) if '.png' in input_file]

# match_images(files, input_dir)
# non_match_images(files, input_dir)

In [58]:
def encode_images(input_dir, output_filename, ext='.png'):
    # Ensure the input directory has a trailing slash
    if input_dir[-1] != '/':
        input_dir += '/'

    fs = [input_dir + x for x in np.sort(os.listdir(input_dir)) if ext in x]
    num_imgs = len(fs)
    output_file = open(output_filename, "wb")
    print(num_imgs)
    # Write items in the header
    # MNIST uses 2051 as a magic number in the header, so following convention here
    output_file.write(struct.pack('>i', 2051))       # Magic Number for train/test images
    output_file.write(struct.pack('>i', num_imgs))   # Number of images

    # Load the first image to get dimensions
    im = np.asarray(Image.open(fs[0]).convert('L'), dtype=np.uint32)
    r,c = im.shape

    # Write the rest of the header
    output_file.write(struct.pack('>i', r))          # Number of rows in 1 image
    output_file.write(struct.pack('>i', c))          # Number of columns in 1 image

    # For each image, record the pixel values in the binary file
    for img in range(10): #num_imgs
        im = np.asarray(Image.open(fs[img]).convert('L'), dtype=np.uint32)
        for i in range(im.shape[0]):
            for j in range(im.shape[1]):
                output_file.write(struct.pack('>B', im[i,j]))

    # Close the file
    output_file.close()
    
#     with open(output_filename, encoding=utf-8) as f_in:
#         print(enc, f.read(500))
#     f_in = open(output_filename)
    f_in = codecs.open(output_filename,"r",encoding="utf-8")
    f_out = gzip.open(output_filename + '.gz', 'wb')
    f_out.writelines(f_in)
#     f_out.writelines(f_in)
    f_out.close()
    f_in.close()
    os.remove(output_filename)
    

#############################encoding labels the same way as images#################################################################################################################

def encode_labels(input_dir, output_filename, ext='.png' ):

    if input_dir[-1] != '/':
        input_dir += '/'

    fs = [input_dir + x for x in np.sort(os.listdir(input_dir)) if ext in x]
    num_imgs = len(fs)
    output_file = open(output_filename, "wb")

    print (num_imgs)
    # Write items in the header
    # MNIST uses 2051 as a magic number in the header, so following convention here
    output_file.write(struct.pack('>i', 2049))  # Magic Number for train/test images
    output_file.write(struct.pack('>i', num_imgs))  # Number of images

    # Load the first image to get dimensions
    print(fs[0])
    im = np.asarray(Image.open(fs[0]).convert('L'), dtype=np.uint32)

    # For each image, record the pixel values in the binary file
    count = 0
    for img in range(num_imgs):
        count+=1
        print(count)
        #print fs[img]
        if re.match(input_dir+'match.*', fs[img]):
            label = 1
        elif re.match(input_dir + 'mis_match.*' , fs[img]):
            label=0
        label = np.uint32(label)
        print(label)
        output_file.write(struct.pack('>B', label))

    # Close the file
    output_file.close()

#     f_in = open(output_filename)
#     with open(output_filename, encoding=utf-8) as f_in:
#         print(enc, f.read(500))
    f_in = codecs.open(output_filename,"r",encoding="utf-8")
    f_out = gzip.open(output_filename + '.gz', 'wb')
    f_out.writelines(f_in)
    f_out.close()
    f_in.close()
    os.remove(output_filename)
    
    

In [59]:
encode_images('/Users/nikhilshekhar/deep_learning_working_directory/data/' , '/Users/nikhilshekhar/deep_learning_working_directory/data/training-images-and-ubyte')
encode_labels('/Users/nikhilshekhar/deep_learning_working_directory/data/' , '/Users/nikhilshekhar/deep_learning_working_directory/data/training-labels-and-ubyte')
#encode('/home/sanjana/Handwritten_and_preprocessed_v2/testing_data/' , 'testing-images-and-ubyte',label=False)
#encode('/home/sanjana/Handwritten_and_preprocessed_v2/testing_data/' , 'testing-labels-and-ubyte',label = True)

167211


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8d in position 6: invalid start byte